# Toy Latent Variable Model
Consider a simple latent variable model where we have observations $y_1, \dots, y_n \in \mathbb{R}^d$ drawn from
    \begin{align*}
        z|\theta &\sim \mathcal{N}(\cdot;\theta \textsf{1}_d, \textsf{Id}_d)\\
        y|z &\sim \mathcal{N}(\cdot;z, \textsf{Id}_d)
    \end{align*}
    where $\theta\in\mathbb{R}$ and $\textsf{1}_d, \textsf{Id}_d$ denote the $d$-dimensional vector of ones and the $d\times d$ identity matrix respectively.

In [36]:
# standard libraries
import importlib
from matplotlib import pyplot as plt
import numpy as np
from scipy.stats import norm
from scipy import linalg, stats, optimize
import time
import sys
sys.path.append('/Users/francescacrucinio/Documents/MD_LVM')

# modules from particles
import particles  # core module
from particles import smc_samplers as ssp
from particles import distributions as dists  # where probability distributions are defined
from particles import resampling as rs


import pgd
import md_lvm
importlib.reload(pgd)
importlib.reload(md_lvm)

<module 'md_lvm' from '/Users/francescacrucinio/Documents/MD_LVM/md_lvm.py'>

## Data

In [19]:
D = 50  # Dimensionality of latent variables.
thdata = 1  # Parameter value used to generate the data.

# Generate the data:
y = np.random.normal(0, 1, (D, 1)) + np.random.normal(thdata, 1, (D, 1))

In [20]:
# get MLE
theta_mle = np.mean(y)
theta_mle

0.6096408318585083

## Compare PGD, IPLA and MD with different stepsizes

In [47]:
#Set approximation parameters:
Niter = 1500  # Number of steps.
N = 1000  # Number of particles.

th0 = np.array([0.0])  # Initial parameter guess.
X0 = np.random.normal(size = (D, N))  # Initial particle cloud.

In [48]:
optimal_stepsize = 2/(2+D)
optimal_stepsize

0.038461538461538464

### Optimal

In [49]:
# # Run the algorithms using the optimal step-sizes:
# start = time.time()
# th_pgd, X_pgd = pgd.pgd_tg(y, optimal_stepsize, Niter, N, th0, X0)
# end = time.time()
# print(end-start)
# start = time.time()
# th_ipla, X_ipla = pgd.ipla_tg(y, optimal_stepsize, Niter, N, th0, X0)
# end = time.time()
# print(end-start)
# start = time.time()
# th_md, X_md, W_md = md_lvm.md_toy_lvm(y, optimal_stepsize, Niter, N, th0, X0.T)
# end = time.time()
# print(end-start)
start = time.time()
th_md2, X_md2, W_md2 = md_lvm.md_toy_lvm_fast(y, optimal_stepsize, Niter, N, th0, X0.T)
end = time.time()
print(end-start)

LinAlgError: 33-th leading minor of the array is not positive definite

In [ ]:
# Plot parameter estimates as a function of step number k:
plt.plot(th_pgd, label='PGD', alpha = 0.5)
plt.plot(th_ipla, label='IPLA', alpha = 0.5)
plt.plot(th_md, label='MD', alpha = 0.5)
plt.plot(th_md2, label='SMC', alpha = 0.5)
plt.plot(y.mean()*np.ones(Niter), label='Optimal theta', c = 'black', linestyle = 'dashed', lw = 2)
plt.legend(loc='lower right')
plt.ylim([0, 1.2*y.mean()])
plt.xlim([0, Niter])
# plt.savefig('toy_optimal_gamma.pdf', bbox_inches="tight")

In [ ]:
fig, (ax1, ax2) = plt.subplots(2)
ax1.hist(X_pgd[0, :], density = True, bins = 50, label='PGD');
ax1.hist(X_ipla[0, :], density = True, bins = 50, label='IPLA');
ax1.plot(np.linspace(-5, 5, 100), norm.pdf(np.linspace(-5, 5, 100), scale = np.sqrt(0.5), loc = 0.5*(np.mean(y)+1)), label = 'True posterior')
ax1.legend()
ax2.hist(X_md[Niter-1, :, 0], density = True, bins = 50, weights = W_md, label='MD');
ax2.hist(X_md2[Niter-1, :, 0], density = True, bins = 50, weights = W_md2, label='MD');
ax2.plot(np.linspace(-5, 5, 100), norm.pdf(np.linspace(-5, 5, 100), scale = np.sqrt(0.5), loc = 0.5*(np.mean(y)+1)), label = 'True posterior')
ax2.legend()